In [16]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import pprint
import pandas as pd
import numpy as np
import itertools
import os

In [17]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.realtor_db
collection = db.homes

In [23]:
# URL of page to be scraped
url = 'https://www.realtor.com/realestateandhomes-search/08468'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')
soup

<!DOCTYPE html>
<html lang="en"><head><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" name="viewport"/><meta content="#d92228" name="theme-color"/><meta content="Search real estate property records, houses, condos, land and more on realtor.com&amp;reg;. Find property info from the most comprehensive source of home data online." name="description"/><meta content="35368227870" property="fb:pages"/><meta content="authenticity_token" name="csrf-param"/>
<meta content="kgq0PoXvKmAjRgVjckeX0cYTCnU3aB5OCzwiXClxTpzkIHFpPMvO5jM2uOyz54dILiEQyrV1stKpDhtn+ma3YQ==" name="csrf-token"/><title>Find Real Estate, Homes for Sale, Apartments &amp; Houses for Rent - realtor.comÂ®</title>
<style type="text/css">
  hr,img{border:0}.list-horizontal,.list-inline,.list-unstyled{padding-left:0;list-style:none}.form-control,.global-login-version-2 .global-login-form-control{-o-transition:border-color ease-in-out .15s,box-

In [24]:
# Retrieve the parent divs for all articles
#results = soup.find_all('li', class_='component_property-card js-component_property-card js-quick-view')
#results = soup.find_all('li', class_='component_property-card', data-url="/realestateandhomes-detail/Pebble-Beach_Venue-at-Cobblestone-Creek_2170-Lawrenceville-Rd_Lawrence-Township_NJ_08648_P414997639822")
results = soup.find_all(class_="detail-wrap")

print(results)

[]


In [6]:
# Loop through results to retrieve article title, header, and timestamp of article
df_list = []

for result in results:
    address = result.find(class_='listing-street-address').text
    zipcode = result.find(class_='listing-postal').text
    price = result.find(class_='data-price').text

    # Dictionary to be inserted into MongoDB
    home = {
        'zipcode': zipcode,
        'address': address,
        'price': price
    }

    # Insert dictionary into MongoDB as a document
    collection.insert_one(home)
    df_list.append(home)
    #print(home)


home_df = pd.DataFrame(df_list)
home_df

,_id,address,price,zipcode
0,5c6ddd5485949423a0581e18,63 Giggleswick Way,"$524,900",08820
1,5c6ddd5485949423a0581e19,90 Pheasant Run,"$375,000",08820
2,5c6ddd5485949423a0581e1a,55 Gate House Ln,"$514,900",08820
3,5c6ddd5485949423a0581e1b,408 Westgate Dr,"$399,999",08820
4,5c6ddd5485949423a0581e1c,85 Westgate Dr,"$425,000",08820
5,5c6ddd5485949423a0581e1d,15 Dogwood Dr,"$350,000",08820
6,5c6ddd5485949423a0581e1e,12 Cactus Ct,"$479,900",08820
7,5c6ddd5485949423a0581e1f,23 Lynnwood Rd,"$574,900",08820
8,5c6ddd5485949423a0581e20,536 Westgate Dr,"$419,000",08820
9,5c6ddd5485949423a0581e21,26 Cottonwood Ct,"$639,000",08820


In [18]:
income = pd.read_csv("MedianIncomeZip.csv")

income_df = pd.DataFrame(income)
income_df = income_df.loc[income_df["Zip"] > 0]
income_df["Zip"] = income_df["Zip"].astype('int')
income_df.rename(columns = {' HC03_EST_VC02 Median income (dollars); Estimate; Households ': "Income"}, inplace = True)
#Cleans out 
income_df['Income'] = income_df['Income'].str.lstrip(" $")
income_df['Income'] = income_df['Income'].str.rstrip(" ")

income_df["Income"] = income_df["Income"].replace( '[,]','', regex=True )
income_df["Income"] = income_df["Income"].astype("float")
income_df.head()

KeyError: 'Zip'

In [22]:
#headers = ['Geo','zipcode','income']
#dtypes = [str, str, float]

income = pd.read_csv("zip_income.csv", low_memory=False)
income_df= pd.DataFrame(income)
#income_df['zipcode'].head()
income_df.head()

#home_income_df = pd.merge(home_df, income_df, how='left', on = "zipcode")
data = pd.ExcelWriter('output.xlsx')
home_df.to_excel(data,'Sheet2')

data.save()


In [6]:
# Display the MongoDB records created above
articles = db.articles.find()
for article in articles:
    print(article)

{'_id': ObjectId('5ada7a1bee61f93d3f7e3d07'), 'title': 'Lady Byng Trophy finalists unveiled', 'lede': "Barkov, O'Reilly, William Karlsson up for award given for combination of skill, gentlemanly play", 'date': '2018-04-20', 'time published': '19:00pm'}
{'_id': ObjectId('5ada7a1bee61f93d3f7e3d08'), 'title': 'Peters resigns as Hurricanes coach', 'lede': 'Went 137-138-53 in four seasons with Carolina', 'date': '2018-04-20', 'time published': '12:00am'}
{'_id': ObjectId('5ada7a1bee61f93d3f7e3d09'), 'title': 'Bergeron skates, could return for Bruins in Game 5 against Maple Leafs', 'lede': 'Boston one win from advancing to second round', 'date': '2018-04-20', 'time published': '14:33pm'}
{'_id': ObjectId('5ada7a1bee61f93d3f7e3d0a'), 'title': 'Friday Four: Golden Knights continue fairy-tale season', 'lede': 'NHL Network analyst Weekes also discusses Sharks, Capitals-Blue Jackets series, Subban', 'date': '2018-04-20', 'time published': '14:01pm'}
{'_id': ObjectId('5ada7a1bee61f93d3f7e3d0b'), '